In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
import re
from typing import Dict, List, Tuple
import numpy as np
from scipy.signal import find_peaks, peak_widths
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import access_token
access_token.log_notebook_usage()

class XRDVisualizationTool:
    def __init__(self):
        self.data_files = {}  # Store parsed data: {filename: (x_data, y_data, metadata)}
        self.checkboxes = {}  # Store checkboxes for each file
        self.individual_plots = {}  # Store individual plot widgets
        self.overlay_plot_output = widgets.Output()
        
        # Color cycle for overlay plot (all solid lines)
        self.colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
        
        # Stagger control
        self.stagger_slider = widgets.FloatSlider(
            value=0.0,
            min=0.0,
            max=1000.0,
            step=10.0,
            description='Stagger Offset:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        self.stagger_slider.observe(self.update_overlay_plot, names='value')
        
        self.setup_ui()
    
    def parse_xy_file(self, file_content: str, filename: str) -> Tuple[List[float], List[float], Dict]:
        """Parse the .xy file format and extract data and metadata"""
        lines = file_content.strip().split('\n')
        
        # Parse metadata from the first line
        metadata = {}
        if lines[0].startswith("'Id:"):
            # Remove quotes and parse key-value pairs
            metadata_line = lines[0].strip("'")
            # Use regex to find key-value pairs
            pattern = r'(\w+):\s*"([^"]*)"'
            matches = re.findall(pattern, metadata_line)
            metadata = dict(matches)
        
        # Parse data points (skip metadata line)
        x_data = []
        y_data = []
        
        for line in lines[1:]:
            if line.strip():  # Skip empty lines
                try:
                    parts = line.split()
                    if len(parts) >= 2:
                        x = float(parts[0])
                        y = float(parts[1])
                        x_data.append(x)
                        y_data.append(y)
                except ValueError:
                    continue  # Skip lines that can't be parsed
        
        return x_data, y_data, metadata
    
    def find_peaks_in_data(self, x_data: List[float], y_data: List[float], 
                          height_threshold: float = None, prominence: float = None) -> Tuple[List[int], List[float], List[float]]:
        """Find peaks in XRD data using scipy's find_peaks"""
        y_array = np.array(y_data)
        x_array = np.array(x_data)
        
        # Set default parameters if not provided
        if height_threshold is None:
            height_threshold = np.max(y_array) * 0.1  # 10% of max intensity
        if prominence is None:
            prominence = np.max(y_array) * 0.05  # 5% of max intensity
        
        # Find peaks
        peaks, properties = find_peaks(y_array, 
                                     height=height_threshold, 
                                     prominence=prominence,
                                     distance=5)  # Minimum distance between peaks
        
        # Get peak positions and intensities
        peak_positions = x_array[peaks].tolist()
        peak_intensities = y_array[peaks].tolist()
        
        return peaks, peak_positions, peak_intensities
    
    def create_peak_controls(self, filename: str) -> widgets.VBox:
        """Create peak detection controls for a file"""
        # Peak detection parameters
        height_slider = widgets.FloatSlider(
            value=10.0,
            min=0.1,
            max=1000.0,
            step=1.0,
            description='Min Height:',
            style={'description_width': '80px'},
            layout=widgets.Layout(width='300px')
        )
        
        prominence_slider = widgets.FloatSlider(
            value=5.0,
            min=0.1,
            max=500.0,
            step=1.0,
            description='Prominence:',
            style={'description_width': '80px'},
            layout=widgets.Layout(width='300px')
        )
        
        show_peaks_checkbox = widgets.Checkbox(
            value=True,
            description='Show Peaks',
            style={'description_width': 'initial'}
        )
        
        peak_info_output = widgets.Output(layout=widgets.Layout(height='100px'))
        
        # Function to update peaks when sliders change
        def update_peaks(change=None):
            self.update_individual_plot_peaks(filename, height_slider.value, 
                                            prominence_slider.value, show_peaks_checkbox.value, 
                                            peak_info_output)
        
        # Observe slider changes
        height_slider.observe(update_peaks, names='value')
        prominence_slider.observe(update_peaks, names='value')
        show_peaks_checkbox.observe(update_peaks, names='value')
        
        # Initial peak detection
        update_peaks()
        
        # Create controls layout
        controls = widgets.VBox([
            widgets.HTML("<b>Peak Detection Controls:</b>"),
            widgets.HBox([height_slider, prominence_slider]),
            show_peaks_checkbox,
            widgets.HTML("<b>Detected Peaks:</b>"),
            peak_info_output
        ])
        
        return controls
    
    def update_individual_plot_peaks(self, filename: str, height_threshold: float, 
                                    prominence: float, show_peaks: bool, peak_info_output: widgets.Output):
        """Update individual plot with peak detection"""
        x_data, y_data, metadata = self.data_files[filename]
        
        # Get the existing plot widget
        plot_widget = self.individual_plots[filename]
        
        # Clear existing traces
        with plot_widget.batch_update():
            plot_widget.data = []
            
            # Add main data trace
            plot_widget.add_scatter(
                x=x_data,
                y=y_data,
                mode='lines',
                name='Data',
                line=dict(width=2, color='blue')
            )
            
            # Find and add peaks if enabled
            peak_info_text = "No peaks detected"
            if show_peaks:
                peaks, peak_positions, peak_intensities = self.find_peaks_in_data(
                    x_data, y_data, height_threshold, prominence
                )
                
                if len(peak_positions) > 0:
                    # Add peak markers
                    plot_widget.add_scatter(
                        x=peak_positions,
                        y=peak_intensities,
                        mode='markers',
                        name='Peaks',
                        marker=dict(
                            color='red',
                            size=8,
                            symbol='triangle-up'
                        ),
                        hovertemplate='Peak at 2θ: %{x:.2f}°<br>Intensity: %{y:.1f}<extra></extra>'
                    )
                    
                    # Create peak info text
                    peak_info_lines = [f"Found {len(peak_positions)} peaks:"]
                    for i, (pos, intensity) in enumerate(zip(peak_positions, peak_intensities)):
                        peak_info_lines.append(f"Peak {i+1}: 2θ = {pos:.2f}°, I = {intensity:.1f}")
                    peak_info_text = "\n".join(peak_info_lines)
        
        # Update peak info output
        with peak_info_output:
            clear_output(wait=True)
            print(peak_info_text)
    
    def create_individual_plot(self, filename: str, x_data: List[float], y_data: List[float], metadata: Dict):
        """Create an individual plot for a file with peak detection"""
        # This will be called by update_individual_plot_peaks initially
        pass
    
    def on_file_upload(self, change):
        """Handle file upload and create individual plots"""
        uploaded_files = change['new']
        
        # Handle the uploaded files - they come as a tuple of file objects
        for file_obj in uploaded_files:
            filename = file_obj.name
            if filename.endswith('.xy'):
                # Parse the file - convert memoryview to bytes then to string
                file_content = bytes(file_obj.content).decode('utf-8')
                x_data, y_data, metadata = self.parse_xy_file(file_content, filename)
                
                # Store the data
                self.data_files[filename] = (x_data, y_data, metadata)
                
                # Create initial plot widget
                fig = go.Figure()
                
                # Create title with metadata info
                title_parts = [f"File: {filename}"]
                if 'Id' in metadata:
                    title_parts.append(f"ID: {metadata['Id']}")
                if 'Operator' in metadata:
                    title_parts.append(f"Operator: {metadata['Operator']}")
                
                fig.update_layout(
                    title='<br>'.join(title_parts),
                    xaxis_title='2θ (degrees)',
                    yaxis_title='Intensity',
                    width=700,
                    height=450,
                    showlegend=True
                )
                
                # Store the plot widget
                plot_widget = go.FigureWidget(fig)
                self.individual_plots[filename] = plot_widget
                
                # Create checkbox
                checkbox = widgets.Checkbox(
                    value=False,
                    description=f'Include {filename}',
                    style={'description_width': 'initial'}
                )
                checkbox.observe(self.update_overlay_plot, names='value')
                self.checkboxes[filename] = checkbox
                
                # Create peak detection controls (this will also create the initial plot)
                peak_controls = self.create_peak_controls(filename)
                self.peak_controls = getattr(self, 'peak_controls', {})
                self.peak_controls[filename] = peak_controls
        
        self.update_display()
    
    def update_overlay_plot(self, change=None):
        """Update the overlay plot based on selected checkboxes"""
        with self.overlay_plot_output:
            clear_output(wait=True)
            
            # Get selected files
            selected_files = [filename for filename, checkbox in self.checkboxes.items() if checkbox.value]
            
            if not selected_files:
                print("No files selected for overlay plot")
                return
            
            # Create overlay plot
            fig = go.Figure()
            
            # Get stagger offset
            stagger_offset = self.stagger_slider.value
            
            for i, filename in enumerate(selected_files):
                x_data, y_data, metadata = self.data_files[filename]
                
                # Apply stagger offset - each subsequent curve is offset upward
                staggered_y_data = [y + (i * stagger_offset) for y in y_data]
                
                # Use only colors - all lines are solid
                color = self.colors[i % len(self.colors)]
                
                # Create display name with offset info if staggered
                display_name = filename
                if stagger_offset > 0:
                    display_name = f"{filename} (+{i * stagger_offset:.0f})"
                
                fig.add_trace(go.Scatter(
                    x=x_data,
                    y=staggered_y_data,
                    mode='lines',
                    name=display_name,
                    line=dict(
                        color=color,
                        width=2
                    )
                ))
            
            # Update layout
            title = 'Overlay Plot - Selected Files'
            if stagger_offset > 0:
                title += f' (Staggered by {stagger_offset})'
            
            fig.update_layout(
                title=title,
                xaxis_title='2θ (degrees)',
                yaxis_title='Intensity',
                width=900,
                height=600,
                showlegend=True,
                legend=dict(
                    yanchor="top",
                    y=0.99,
                    xanchor="left",
                    x=1.01
                )
            )
            
            display(go.FigureWidget(fig))
    
    def update_display(self):
        """Update the entire display with individual plots and checkboxes"""
        with self.main_output:
            clear_output(wait=True)
            
            if not self.data_files:
                print("No files uploaded yet. Please upload .xy files above.")
                return
            
            print(f"Uploaded {len(self.data_files)} files:")
            print("=" * 50)
            
            # Display individual plots with checkboxes
            for filename in self.data_files.keys():
                print(f"\n{filename}:")
                
                # Create horizontal layout: checkbox + plot
                checkbox = self.checkboxes[filename]
                plot = self.individual_plots[filename]
                peak_controls = getattr(self, 'peak_controls', {}).get(filename)
                
                # Display checkbox
                display(checkbox)
                
                # Display plot
                display(plot)
                
                # Display peak controls if available
                if peak_controls:
                    display(peak_controls)
                
                print("-" * 50)
    
    def setup_ui(self):
        """Set up the user interface"""
        # File upload widget
        self.file_upload = widgets.FileUpload(
            accept='.xy',
            multiple=True,
            description='Upload .xy files'
        )
        self.file_upload.observe(self.on_file_upload, names='value')
        
        # Main output area for individual plots
        self.main_output = widgets.Output()
        
        # Instructions
        instructions = widgets.HTML("""
        <h2>XRD Data Visualization Tool</h2>
        <p><strong>Instructions:</strong></p>
        <ol>
            <li>Upload one or more .xy files using the upload button below</li>
            <li>Individual plots will be displayed with checkboxes</li>
            <li>Check the boxes next to plots you want to overlay</li>
            <li>Use the stagger offset slider to vertically separate curves for better visualization</li>
            <li>The overlay plot will update automatically at the bottom</li>
        </ol>
        """)
        
        # Display the interface
        display(instructions)
        display(self.file_upload)
        display(self.main_output)
        
        # Stagger control section
        stagger_section = widgets.HTML("""
        <h3>Overlay Plot Controls:</h3>
        <p>Adjust the stagger offset to vertically separate curves in the overlay plot:</p>
        """)
        
        print("\nOverlay Plot:")
        print("=" * 50)
        display(stagger_section)
        display(self.stagger_slider)
        display(self.overlay_plot_output)
        
        # Initial display
        self.update_display()

# Create and run the tool
tool = XRDVisualizationTool()

HTML(value='\n        <h2>XRD Data Visualization Tool</h2>\n        <p><strong>Instructions:</strong></p>\n   …

FileUpload(value=(), accept='.xy', description='Upload .xy files', multiple=True)

Output()


Overlay Plot:


HTML(value='\n        <h3>Overlay Plot Controls:</h3>\n        <p>Adjust the stagger offset to vertically sepa…

FloatSlider(value=0.0, description='Stagger Offset:', layout=Layout(width='400px'), max=1000.0, step=10.0, sty…

Output()